


# Hitters Veri Seti Üzerinde Maaş Tahmini:


## Veri Setinin Hikayesi:
    
###    "Hitters" Veri seti Amerika'da bir baseball takımının oyuncularının 1986 yılına ait performans ve maaş bilgilerini içeriyor. 
### Orijinal veri seti 322 gözlem ve 20 değişkenden oluşuyor. 
    
### Değişkenler:
**    AtBat:  Oyuncunun 1986 yılında yaptığı toplam vuruşlar.

**    Hits:   Oyuncunun 1986 yılında yaptığı isabetli vuruşlar.

**    HmRun:  Oyuncunun 1986 yılında yaptığı 'Home Run' sayısı. Vurucunun topu oyun alanı dışına yollamasıdır. Bir vurucunun takımına sağlayabileceği en büyük yarar sayı turudur. Çünkü normal bir koşuda bir vurucu ancak bir sayı kazandırabilirken, sayı turunda sahada ne kadar koşucu varsa bir o kadar daha sayı kazandırmış olur.

**    Runs: Oyuncunun 1986 yılında yaptığı 'Run' sayısı. Sayı (Koşucunun oyundan çıkmadan tüm kaleleri dolaşıp sonuçta merkez kale levhasına dokunup sayı yapmasıdır.)

**    RBI:  1986 yılında vuruş yapılan koşu sayısı.

**    Walks: Yürütmek (Kendisine, vuruş bölgesi (Strike Zone) dışına giden dört ball atılan vurucuya verilen birinci kaleye gitme ödülüdür. Bu ödül sonucu söz konusu vurucu, vuruş yapmasına gerek kalmadan birinci kaleye yürür. Bir nevi teknik faul.)

**    Years: Oyuncunun tecrübesi (yıl olarak).

**    ('C' harfi ile başlayan değişkenler, oyuncunun kariyeri boyunca yaptıklarını ifade eder.)

**    League:   1986 sonunda oyuncu ligini gösteren A ve N değerlerine sahip bir faktör.

**    Division:  1986 sonunda oyuncunun ayrılmasını gösteren E ve W değerlerine sahip bir faktör.

**    PutOuts:  Oyuncunun 1986 yılındaki putouts sayısı.

**    Assists: Oyuncunun 1986 yılındaki assist sayısı.

**    Errors: Oyuncunun 1986 yılındaki hata sayısı.

**    Salary: Oyuncunun aktüel maaşı.

**    NewLeague: 1987 başında oyuncunun ligini gösteren A ve N değerlerine sahip bir faktör.
    
    
    
## 1. Gerekli import işlemleri:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR
import xgboost
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

## 2. Veri Setinin Okunup Görüntülenmesi:

In [ ]:
hitters = pd.read_csv('../input/hitters-baseball-data/Hitters.csv')
hitters.head()

In [ ]:
hitters.info()

In [ ]:
hitters.isnull().sum()

### Veri setinde üç tane 'Object' türünde değişken olduğu ve'Salary' değişkeni içerisinde 59 eksik veri bulunduğu gözlemlendi. 

İlk olarak 'get.dummies' işlemi ile 'Object' olarak görülen değişkenlerden kurtuluyoruz. 

In [ ]:
dummies = pd.get_dummies(hitters[['League', 'Division', 'NewLeague']]) 
dummies.head()

In [ ]:
X_ = hitters.drop(['League', 'Division', 'NewLeague'], axis=1).astype('float64') 

hitters = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1) 

hitters.info()

In [ ]:
hitters.describe().T

## 3. Maaş tahmini için uygulayacağımız farklı senaryolar için farklı veri setleri oluşturacağız. 


### 3.1. Değişkenler üzerinde hiçbir değişiklik yapmadan sadece eksik verileri silerek 'df_1' adlı veri setini oluşrutuyoruz:

In [ ]:
df_1 = hitters.dropna()
df_1.head()

In [ ]:
df_1.info()

### 3.2. Eksik verilerin yerine bulundukları 'Salary' değişkeninin ortalaması atanarak 'df_2' adlı veri seti oluşturuluyor:

In [ ]:
df = hitters.copy()
df['Salary'].fillna(df['Salary'].mean(), inplace = True) 
df_2 = df.copy()
df_2.info()

### 3.3.  Eksik Veriler Gradient Boosting Regression Tahmin Sonuçları ile Doldurulup 'df_3' adlı veri seti oluşturuluyor:

In [ ]:
hitters.head()

In [ ]:
null = hitters[hitters['Salary'].isnull()] #Eksik veri barındıran gözlemlerin seçilmesi
null.head()

In [ ]:
df = hitters.dropna() #Eksik veri barındıran gözlemlerin silinmesi
X_train = df.drop('Salary', axis = 1) #Train setinin tanımlanması
X_train.head()

In [ ]:
y_train = df[['Salary']] #Train setinin bağımlı değişkeninin belirlenmesi
y_train.head()

In [ ]:
X_test = null.drop('Salary', axis = 1) # Veri setindeki eksik veri barındıran gözlemlerin test seti olarak tanımlanması
X_test.head()

In [ ]:
gbm_model = GradientBoostingRegressor().fit(X_train, y_train)
gbm_model_pred_test = gbm_model.predict(X_test)
gbm_model_pred_test

In [ ]:
X_test['Salary'] = gbm_model_pred_test

In [ ]:
df_3 = pd.concat([df, X_test], ignore_index = True)
df_3.head()

In [ ]:
df_3.info()

In [ ]:
df_3.describe().T

### 3.4. Eksik Veriler Tahminlerle Doldurulup  Aykırı Değerler Baskılanarak 'df_4' oluşturuluyor:

In [ ]:
df_3.info()

### LocalOutlierFactor ile değişkenlerin aykırı değerleri belirlenecek.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(df_3)
df_scores = clf.negative_outlier_factor_
df_scores[0:20]

In [ ]:
np.sort(df_scores)


In [ ]:
np.sort(df_scores)[16]

In [ ]:
esik_deger = np.sort(df_scores)[16]
esik_deger

In [ ]:
aykiri_df = df_scores > esik_deger


In [ ]:
df_3[df_scores == esik_deger]


In [ ]:
baski_degeri = df_3[df_scores == esik_deger]

In [ ]:
aykirilar = df_3[~aykiri_df] 


In [ ]:
aykirilar.to_records(index=False)

In [ ]:
res = aykirilar.to_records(index=False)


In [ ]:
res[:] = baski_degeri.to_records(index = False)


In [ ]:
naykiri = df_3[aykiri_df]
naykiri.describe().T

In [ ]:
aykiri = pd.DataFrame(res, index = df_3[~aykiri_df].index)
aykiri.describe().T

In [ ]:
df_4 = pd.concat([naykiri, aykiri], ignore_index = True)
df_4.describe().T

### 3.5. Eksik Veriler Tahminlerle Doldurulup Aykırı Veriler Silinerek 'df_5' Oluşturuluyor:

In [ ]:
df_3.info()

In [ ]:
df_5 = df_3[df_scores > esik_deger]
df_5.info()

### 3.6. Eksik Veriler Silinip Aykırı Veriler Baskılanarak 'df_6' Oluşturuluyor:

In [ ]:
df.info()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(df)
df6_scores = clf.negative_outlier_factor_
df6_scores[0:20]

In [ ]:
np.sort(df6_scores)


In [ ]:
np.sort(df6_scores)[8]

In [ ]:
esik_deger6 = np.sort(df6_scores)[8]
esik_deger6

In [ ]:
aykiri_df6 = df6_scores > esik_deger6
aykiri_df6

In [ ]:
df[df6_scores == esik_deger6]


In [ ]:
baski_degeri6 = df[df6_scores == esik_deger6]

In [ ]:
aykirilar6 = df[~aykiri_df6] 


In [ ]:
aykirilar6.to_records(index=False)

In [ ]:
res6 = aykirilar6.to_records(index=False)


In [ ]:
res6[:] = baski_degeri6.to_records(index = False)


In [ ]:
naykiri6 = df[aykiri_df6]
naykiri6.describe().T

In [ ]:
aykiri6 = pd.DataFrame(res6, index = df[~aykiri_df6].index)
aykiri6.describe().T

In [ ]:
df_6 = pd.concat([naykiri6, aykiri6], ignore_index = True)
df_6.describe().T

### 3.7.  Eksik ve Aykırı Veriler Silinip 'df_7' Veri Seti Oluşturuluyor:

In [ ]:
df_7 = naykiri6
df_7.info()

## 4. Değişken Mühendisliği:



In [ ]:
df_8 = hitters.copy()
df_8.info()

### 4.1.  Kategorik değişkenler "League_N, Division_W, NewLeague_N" 

In [ ]:
cat_df = df_8.select_dtypes(include=["uint8"])
cat_df.head()

In [ ]:
print(cat_df.League_N.unique())
print(cat_df["League_N"].value_counts().count())
print(cat_df["League_N"].value_counts())
print(df_8["League_N"].value_counts().plot.barh())
df_8.groupby('League_N')['Salary'].mean()

In [ ]:
print(cat_df.Division_W.unique())
print(cat_df["Division_W"].value_counts().count())
print(cat_df["Division_W"].value_counts())
print(df_8["Division_W"].value_counts().plot.barh())
df_8.groupby('Division_W')['Salary'].mean()

In [ ]:
print(cat_df.NewLeague_N.unique())
print(cat_df["NewLeague_N"].value_counts().count())
print(cat_df["NewLeague_N"].value_counts())
print(df_8["NewLeague_N"].value_counts().plot.barh())
df_8.groupby('NewLeague_N')['Salary'].mean()

In [ ]:
Experience = []
for ex in df_8['Years']:
    if ex < 5:
        Experience.append(1)
    elif (ex >= 5) & (ex < 10):
        Experience.append(2)
    elif (ex >= 10) & (ex < 15):
        Experience.append(3)
    elif (ex >= 15) & (ex < 20):
        Experience.append(4)
    else:
        Experience.append(5)
df_8['Experience'] = Experience

### 'Years' değişkeni 1 ile 24 sayıları arasındaki değerlerden oluşuyor. Bunu 0-4, 5-9, 10-19, 20-24 şeklinde 1'den 5'e kadar rakamlarla numaralandırdık.

In [ ]:
df_8.groupby(['League_N', 'Division_W', 'NewLeague_N'])['Salary'].mean()

In [ ]:
df_8.groupby(['League_N', 'Division_W', 'NewLeague_N', 'Experience'])['Salary'].mean()

### 'League_N', 'Division_W', 'NewLeague_N', 'Experience' değişkenleri groupby yapılarak 'Salary' değşikeninin ortalaması alınıyor ve 'Salary' değişkenindeki eksik değerlerin yerine bu ortalamalar atanıyor.

In [ ]:
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 1), "Salary"] = 145.961538
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 2), "Salary"] = 774.434536
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 3), "Salary"] = 918.073533
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 4), "Salary"] = 614.375000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 2), "Salary"] = 850.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 3), "Salary"] = 833.333333
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 1), "Salary"] = 203.821429
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 2), "Salary"] = 528.108696
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 3), "Salary"] = 786.916700
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 4), "Salary"] = 479.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 1), "Salary"] = 96.666667
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 0) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 3), "Salary"] = 825.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 1), "Salary"] = 70.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 2), "Salary"] = 525.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 3), "Salary"] = 500.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 4), "Salary"] = 1050.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 1), "Salary"] = 313.753320
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 2), "Salary"] = 776.095190
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 3), "Salary"] = 949.010143
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 0) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 4), "Salary"] = 486.111000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 1), "Salary"] = 565.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 2), "Salary"] = 405.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 0) & (df_8['Experience'] == 3), "Salary"] = 250.000000
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 1), "Salary"] = 188.138889
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 2), "Salary"] = 538.114053
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 3), "Salary"] = 723.452429
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 4), "Salary"] = 763.666600
df_8.loc[(df_8["Salary"].isnull()) & (df_8["League_N"] == 1) & (df_8['Division_W'] == 1) & (df_8["NewLeague_N"] == 1) & (df_8['Experience'] == 5), "Salary"] = 475.000000


In [ ]:
df_8.info()

### 4.2. Değişken Ekleme:

Veri setinde oyuncuların 1986 yılındaki ve tüm kariyerleri boyunca elde ettikleri veriler ile kaç yıllık tecrübelerinin olduğu bilgileri yer alıyor. Bu verilerin yıllık ortalamalırını ve 1986 yılındaki verilerin tüm performansa oranını ekliyoruz.  

In [ ]:
df_8['AtBat_rate'] = df_8["CAtBat"] / df_8["Years"]
df_8['Hits_rate'] = df_8["CHits"] / df_8["Years"]
df_8['HmRun_rate'] = df_8["CHmRun"] / df_8["Years"]
df_8['Runs_rate'] = df_8["CRuns"] / df_8["Years"]
df_8['RBI_rate'] = df_8["CRBI"] / df_8["Years"]
df_8['Walks_rate'] = df_8["CWalks"] / df_8["Years"]

df_8['1986_AtBat_rate'] = df_8["AtBat"] / df_8["CAtBat"]
df_8['1986_Hits_rate'] = df_8["Hits"] / df_8["CHits"]
df_8['1986_HmRun_rate'] = df_8["HmRun"] / df_8["CHmRun"]
df_8['1986_Runs_rate'] = df_8["Runs"] / df_8["CRuns"]
df_8['1986_RBI_rate'] = df_8["RBI"] / df_8["CRBI"]
df_8['1986_Walks_rate'] = df_8["Walks"] / df_8["CWalks"]



In [ ]:
df_8.info()

In [ ]:
df_8 = df_8.dropna()

In [ ]:
df_8.info()


## 5. Tahmin:

### Elimizde 8 adet veri var. Bunların her biri için aşağıdaki modellerle tamin yapılacak. 


### Modeller:

          Linear Regression
          Ridge Regression
          Lasso Regression
          ElasticNet Regression
          LightGBM Regression
          XGBoost Regression
          GradientBoosting Regression 
          RandomForest Regression 
          DecisionTree Regression
          MLP Regression
          KNeighbors Regression
          SupportVector Regression
          

### İlk önce hiperparametre optimizasyonu yapmadan tahminler yapılacak.
### Veri setleri '%80 train seti' ve '%20 test seti' olmak üzere ikiye ayrılacak ve 'random_state = 46' olarak ayarlanacak.


In [ ]:
def compML(df, y, alg):
    #train-test ayrimi
    y = df[y]
    X = df.drop('Salary', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=46)
    #modelleme
    model = alg().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    model_ismi = alg.__name__
    print("  veri seti için  ", model_ismi, "  Modeli Test Hatası:   ",RMSE)
    
    

In [ ]:
models = [LinearRegression,
          Ridge,
          Lasso,
          ElasticNet,
          LGBMRegressor, 
          XGBRegressor, 
          GradientBoostingRegressor, 
          RandomForestRegressor, 
          DecisionTreeRegressor,
          MLPRegressor,
          KNeighborsRegressor, 
          SVR]


In [ ]:
for i in models:
    compML(df_1, "Salary", i)

In [ ]:
for i in models:
    compML(df_2, "Salary", i)

In [ ]:
for i in models:
    compML(df_3, "Salary", i)

In [ ]:
for i in models:
    compML(df_4, "Salary", i)

In [ ]:
for i in models:
    compML(df_5, "Salary", i)

In [ ]:
for i in models:
    compML(df_6, "Salary", i)

In [ ]:
for i in models:
    compML(df_7, "Salary", i)

In [ ]:
for i in models:
    compML(df_8, "Salary", i)

### Hiperparametre optimizasyonu yapılmadan gerçekleştirilen tahminlerde yukarıdaki sonuçlar alınmıştır. 




# 6. Hiperparametre Optimizasyonları



### Yapılan ilk tahmin işleminde en başarılı olan veri seti df_4 idi. Bu yüzden hiperparametre optimizasyon işlemleri bu veri seti üzerinde yapılacak. 




## 6.1. KNN

In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [ ]:
knn = KNeighborsRegressor()
knn_params = {"n_neighbors": np.arange(1,30,1)}

In [ ]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10).fit(X_train, y_train)

In [ ]:
knn_cv_model.best_params_

In [ ]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"]).fit(X_train, y_train)

In [ ]:
knn_tuned_y_pred = knn_tuned.predict(X_test)

In [ ]:
knn_tuned_RMSE = np.sqrt(mean_squared_error(y_test, knn_tuned_y_pred))
knn_tuned_RMSE

## 6.2. Destek Vektör Regresyonu (Support Vector Regression)



In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
svr_model = SVR("linear") 

In [ ]:
svr_params = {"C": [0.1,0.5,1,3]}

In [ ]:
svr_cv_model = GridSearchCV(svr_model, svr_params, cv = 5, verbose = 2, n_jobs = -1).fit(X_train, y_train)

In [ ]:
svr_cv_model.best_params_

In [ ]:
svr_tuned = SVR("linear", C = 3).fit(X_train, y_train)

In [ ]:
svr_model_y_pred = svr_tuned.predict(X_test)

In [ ]:
svr_model_tuned_RMSE = np.sqrt(mean_squared_error(y_test, svr_model_y_pred))
svr_model_tuned_RMSE

## 6.3. Yapay Sinir Ağları




In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [ ]:
mlp_model = MLPRegressor().fit(X_train_scaled, y_train)

In [ ]:
mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001, 0.0001], 
             "hidden_layer_sizes": [(10,20), (5,5), (100,100)]}

In [ ]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10, verbose = 2, n_jobs = -1).fit(X_train_scaled, y_train)

In [ ]:
mlp_cv_model.best_params_

In [ ]:
mlp_tuned = MLPRegressor(alpha = 0.001, hidden_layer_sizes = (100,100)).fit(X_train_scaled, y_train)

In [ ]:
mlp_y_pred = mlp_tuned.predict(X_test_scaled)

In [ ]:
mlp_tuned_RMSE = np.sqrt(mean_squared_error(y_test, mlp_y_pred))
mlp_tuned_RMSE

## 6.4. CART (Classification and Regression Tree)




In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
cart_model = DecisionTreeRegressor()

In [ ]:
cart_model.fit(X_train, y_train)

In [ ]:
cart_params = {"max_depth": [2,3,4,5,10,20],
              "min_samples_split": [2,10,5,30,50,10]}

In [ ]:
cart_cv_model = GridSearchCV(cart_model, cart_params, cv = 10, verbose = 2, n_jobs = -1).fit(X_train, y_train)

In [ ]:
cart_cv_model.best_params_

In [ ]:
cart_tuned = DecisionTreeRegressor(max_depth = 4, min_samples_split = 2).fit(X_train, y_train)

In [ ]:
cart_model_y_pred = cart_tuned.predict(X_test)
cart_tuned_RMSE = np.sqrt(mean_squared_error(y_test, cart_model_y_pred))
cart_tuned_RMSE

## 6.5. Random Forests




In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
rf_model = RandomForestRegressor(random_state = 46).fit(X_train, y_train)
rf_model

In [ ]:
rf_params = {"max_depth": [5,8,10],
            "max_features": [2,5,10],
            "n_estimators": [200, 500, 1000, 2000],
            "min_samples_split": [2,10,80,100]}

In [ ]:
rf_cv_model = GridSearchCV(rf_model, rf_params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

In [ ]:
rf_cv_model.best_params_

In [ ]:
rf_model = RandomForestRegressor(random_state = 46, 
                                 max_depth = 8,
                                max_features = 5,
                                min_samples_split = 2,
                                 n_estimators = 500)
rf_tuned = rf_model.fit(X_train, y_train)

In [ ]:
rf_y_pred = rf_tuned.predict(X_test)
rf_tuned_RMSE = np.sqrt(mean_squared_error(y_test, rf_y_pred))
rf_tuned_RMSE

## Değişken Önem Düzeyi

In [ ]:
rf_tuned.feature_importances_*100

In [ ]:
Importance = pd.DataFrame({'Importance':rf_tuned.feature_importances_*100}, 
                          index = X_train.columns)


Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', )

plt.xlabel('Variable Importance')
plt.gca().legend_ = None

##  6.6. Gradient Boosting Machines





In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
gbm_model = GradientBoostingRegressor().fit(X_train, y_train)
gbm_model

In [ ]:
gbm_params = {"learning_rate": [0.001,0.1,0.01],
             "max_depth": [3,5,8],
             "n_estimators": [100,200,500],
             "subsample": [1,0.5,0.8],
             "loss": ["ls","lad","quantile"]}

In [ ]:
gbm_cv_model = GridSearchCV(gbm_model, 
                            gbm_params, 
                            cv = 10, 
                            n_jobs=-1, 
                            verbose = 2).fit(X_train, y_train)

In [ ]:
gbm_cv_model.best_params_

In [ ]:
gbm_tuned = GradientBoostingRegressor(learning_rate = 0.1,
                                     loss = "lad",
                                     max_depth = 3,
                                     n_estimators = 100,
                                     subsample = 1).fit(X_train, y_train)

In [ ]:
gbm_tuned_y_pred = gbm_tuned.predict(X_test)
gbm_tuned_RMSE = np.sqrt(mean_squared_error(y_test, gbm_tuned_y_pred))
gbm_tuned_RMSE

## Değişken Önem Düzeyi:

In [ ]:
Importance = pd.DataFrame({'Importance':gbm_tuned.feature_importances_*100}, 
                          index = X_train.columns)


Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', )

plt.xlabel('Variable Importance')
plt.gca().legend_ = None

## 6.7. XGBoost




In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
xgb = XGBRegressor()
xgb

In [ ]:
xgb_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.4,0.7,1]}

In [ ]:
xgb_cv_model  = GridSearchCV(xgb,xgb_params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

In [ ]:
xgb_cv_model.best_params_

In [ ]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.4, 
                         learning_rate = 0.1, 
                         max_depth = 4, 
                         n_estimators = 100).fit(X_train, y_train)

In [ ]:
xgb_tuned_y_pred = xgb_tuned.predict(X_test)
xgb_tuned_RMSE = np.sqrt(mean_squared_error(y_test, xgb_tuned_y_pred))
xgb_tuned_RMSE

## 6.8.  LightGBM




In [ ]:
df_4.head()

In [ ]:
y = df_4['Salary']
X = df_4.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
lgb_model = LGBMRegressor()
lgb_model

In [ ]:
lgbm_params = {"learning_rate": [0.01, 0.1, 0.5, 1],
              "n_estimators": [20,40,100,200,500,1000],
              "max_depth": [1,2,3,4,5,6,7,8,9,10]}

In [ ]:
lgbm_cv_model = GridSearchCV(lgb_model, 
                             lgbm_params, 
                             cv = 10, 
                             n_jobs = -1, 
                             verbose =2).fit(X_train, y_train)

In [ ]:
lgbm_cv_model.best_params_

In [ ]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.1, 
                          max_depth = 2, 
                          n_estimators = 200).fit(X_train, y_train)

In [ ]:
lgbm_tuned_y_pred = lgbm_tuned.predict(X_test)
lgbm_tuned_RMSE = np.sqrt(mean_squared_error(y_test, lgbm_tuned_y_pred))
lgbm_tuned_RMSE

## 6.9.  CatBoost

In [ ]:
cat_df = df_4

In [ ]:
y = cat_df['Salary']
X = cat_df.drop('Salary', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

In [ ]:
catb_model = CatBoostRegressor()

In [ ]:
catb_params = {"iterations": [200,500,100],
              "learning_rate": [0.01,0.1],
              "depth": [3,6,8]}

In [ ]:
catb_cv_model = GridSearchCV(catb_model, 
                           catb_params, 
                           cv = 5, 
                           n_jobs = -1, 
                           verbose = 2).fit(X_train, y_train)

In [ ]:
catb_cv_model.best_params_

In [ ]:
catb_tuned = CatBoostRegressor(depth = 6, iterations = 500, learning_rate = 0.01).fit(X_train, y_train)

In [ ]:
catb_tuned_y_pred = catb_tuned.predict(X_test)

In [ ]:
catb_tuned_RMSE = np.sqrt(mean_squared_error(y_test, catb_tuned_y_pred))
catb_tuned_RMSE

# SONUÇ:

### 'Hitters' veri seti üzerinde yapılan 'Maaş Tahmin' çalışmasında toplam 8 tane veri seti oluşturulmuştur:

### df_1: Hitters veri setindeki eksik verilerin bulunduğu gözlemler silinerek oluşturulmuştur.

### df_2: Hitters veri setindeki eksik veriler bulundukları 'Salary' değişkeninin ortalaması ile doldurularak oluşturulmuştur.

### df_3: Hitters veri setindeki eksik veriler GradientBoostingMashine modeli ile tahmin edilerek doldurulmuştur.

### df_4: df_3 veri setindeki LocalOutlierFactor ile belirlenen aykırı değerler baskılanarak oluşturulmuştur.

### df_5: df_3 veri setindeki LocalOutlierFactor ile belirlenen aykırı değeler silinerek oluşturulmuştur.

### df_6: Hitters veri setindeki eksik verilerin bulunduğu gözlemler silinip ayrıkı değerler baskılanarak oluşturulmuştur.

### df_7: Hitters veri setindeki eksik veriler ve aykırı değerler silinerek oluşturulmuştur.

### df_8: Veri setine yeni değişkenler eklenerek oluşturulmuştur. Years değişkenindeki değerler bölünmüş, 'Experience' değişkeni oluşturulmuş, oyuncuların performanslarının yıllık ortalaması ile 1986 yılındaki performanslarının tüm kariyerlerine oranı değişken olarak eklenmiştir. 



## Daha sonra tüm 'Regresyon Modelleri'için bir fonksiyon yazılmış ve veri setleri üzerinde tek tek tüm modellerle tahmin işlemi yapılmıştır.

## Son olarak Hiperparametre optimizasyonları ile model tuning işlemleri yapılmış ve final modelleri kurulmuştur. 
